In [1]:
google_colab = False

if google_colab:
    !pip install geopandas
    from google.colab import drive
    drive.mount("/content/drive")
    os.chdir('/content/drive/MyDrive/ASU - Zhiang/Projects/instance segmentation/')
    os.listdir()
    
import os

download the rock data: https://drive.google.com/file/d/14L30dFGVBi_iK95BcqAOTi6SGjq59orS/view?usp=sharing

# Annotation map splitting

In [2]:
from rsisa.annotation_map_split import Tile_Splitter
from rsisa.annotation_map_split import Dataset
from fiona.crs import CRS

In [3]:
help(Tile_Splitter)

Help on class Tile_Splitter in module rsisa.annotation_map_split:

class Tile_Splitter(builtins.object)
 |  Tile_Splitter(shapefile_path, save_dir, crs, area_x1=0, area_y1=0, area_x2=30, area_y2=30, tile_size=10, overlap=1, tif_path=None, keep_instance_tif=True)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, shapefile_path, save_dir, crs, area_x1=0, area_y1=0, area_x2=30, area_y2=30, tile_size=10, overlap=1, tif_path=None, keep_instance_tif=True)
 |      Split an annotation map to annotation tiles (shapefiles). It provides an option to split the corresponding tif map. 
 |      
 |      Args:
 |          shapefile_path (string): path of the shapefile to be split
 |          save_dir (string): path of the split shapefiles to be saved
 |          crs (_type_): coordinate reference system (e.g., from fiona.crs import from_epsg)
 |          area_x1 (float): study area coordinate x1 in meters. Not required if tif_path is not None. (x1, y1) are small coordinates; (x2, y2) are large co

In [ ]:
shp_path = '/root/rsisa/data/rocks/rocks.shp'
split_shp_tif_dir = '/root/rsisa/data/rocks/split_shp_tif'
tif_path = '/root/rsisa/data/rocks/rocks.tif'
save_sub_dir = '/root/rsisa/data/rocks/'
tile_length = 10
overlap = 1
ellipse_max = 1.0
pixel_size = 0.00728542 


epsg = 32611  # WGS 84 / UTM zone 11N
crs = CRS.from_epsg(epsg)

print(shp_path)
print(split_shp_tif_dir)
print(tif_path)
print(save_sub_dir)

In [ ]:
# split the study area and create annotation tiles
split_config = {'shapefile_path': shp_path, 
                'save_dir': split_shp_tif_dir, 
                'crs':crs, 
                'tile_size': tile_length, 
                'overlap': overlap,
                'tif_path': tif_path, 
                'keep_instance_tif': True}

ts = Tile_Splitter(**split_config)
ts.split()

In [ ]:
# crop the study area (remove edges)
tile_number = 31
split_config = {'shapefile_path': shp_path, 
                'save_dir': save_sub_dir, # this should be sub dir
                'crs':crs, 
                'tile_size':(tile_length - overlap) * tile_number + overlap + ellipse_max/2., # just change this to the size of study area. here because the ellipses go slightly than the study area, if we set tile_size of 30, the exceeding part will be split into other tiles. thus we have a large tile_size here.
                'overlap':0,
                'tif_path': tif_path, 
                'keep_instance_tif': True}

ts = Tile_Splitter(**split_config)
ts.split()

In [ ]:
# check the generated files
os.listdir('/root/rsisa/data/random_generation_tutorial')

In [ ]:
# create prediction tiles using the annotation tiles
# help(Dataset)

rock_data = Dataset(pixel_size=int(tile_length/pixel_size), split_path=split_shp_tif_dir, input_channel=(0,1,2)) 

In [ ]:
rock_data.save_pickles(zip=zip)

In [ ]:
# check generated prediction tiles (.zip files)
os.listdir('/root/rsisa/data/rocks/split_shp_tif')[:20]

`rocks_time_space.npy` logs the time and space usage of the annotation splitting process

In [ ]:
print(len(rock_data))

In [ ]:
# visualize the dataset

image, target = rock_data[0]
print(target['image_name'])
rock_data.show(0)
print('-')
rock_data.show_overlay(0)
print('-')
rock_data.show_overlay(0, random_color=True, bbox=True)

In [ ]:
# make sure the images at the tif map edge are also checked

for i, (image, target) in enumerate(rock_data):
    if target is not None:
        if target['image_name'].split('/')[-1] == '28_5.tif':
            print(i)
            break

rock_data.show_overlay(i, random_color=True, bbox=True)

# Instance registration without instance segmentation

In [4]:
from rsisa.instance_registration import Instance_Registration

In [5]:
help(Instance_Registration)

Help on class Instance_Registration in module rsisa.instance_registration:

class Instance_Registration(builtins.object)
 |  Instance_Registration(instance_dir, save_shapefile, tif_height_pixel=1000, tif_width_pixel=1000, tif_height_res=-0.01, tif_width_res=0.01, tif_map_file=None, tile_overlap_ratio=0.1, detection_threshold=0.75, segmentation_threshold=0.5, iou_threshold=0.5, disable_merge=False, test=True, unzip=False)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, instance_dir, save_shapefile, tif_height_pixel=1000, tif_width_pixel=1000, tif_height_res=-0.01, tif_width_res=0.01, tif_map_file=None, tile_overlap_ratio=0.1, detection_threshold=0.75, segmentation_threshold=0.5, iou_threshold=0.5, disable_merge=False, test=True, unzip=False)
 |      merge and register instances using split pickle files and tiff tiles. The pickle file name is consistant with the names of its corresponding shapefile and tif file. Each pickle file includes a dictionary of instance predictions, e.g.,

In [14]:
instance_dir = "/root/rsisa/data/rocks/split_shp_tif"
save_shp_file = "/root/rsisa/data/rocks/rocks_merge.shp"
tif_map = "/root/rsisa/data/rocks/0_0.tif"
iou_threshold = 0.5

In [15]:
ir = Instance_Registration(instance_dir, 
                           save_shp_file,
                           tif_width_pixel=tile_length/pixel_size,
                           tif_height_pixel=tile_length/pixel_size,
                           tif_width_res=pixel_size,
                           tif_height_res=-pixel_size,
                           tif_map_file=tif_map,
                           tile_overlap_ratio=overlap/tile_length,
                           iou_threshold=iou_threshold, 
                           disable_merge=False, 
                           unzip=zip)




In [16]:
updated_tile_files, timestamps = ir.start_registration()
ir.combine_shapefiles()

Instance registration: 


100%|█████████████████████████████████████████| 360/360 [28:10<00:00,  4.70s/it]


In [19]:
# check the generated files
os.listdir('/root/rsisa/data/rocks')

['rocks_merge.prj',
 'rocks_merge.shp',
 'rocks.cpg',
 'rocks_merge_remaining.cpg',
 'rocks_merge_twins.pickle',
 'rocks.sbx',
 'rocks_merge_time_space.npy',
 'split_shp_tif',
 'rocks_merge_remaining.dbf',
 'rocks.xml',
 'rocks_time_space.npy',
 'rocks_merge.cpg',
 '0_0.cpg',
 '0_0.tif',
 'rocks.dbf',
 '0_0.prj',
 'rocks_merge_results.npy',
 '0_0.shx',
 'rocks_merge_remaining.prj',
 '0_0.shp',
 'rocks.sbn',
 'rocks_merge_remaining.shp',
 'rocks_merge_remaining.shx',
 'rocks.tif',
 'rocks.shp',
 'rocks.shx',
 '0_0.dbf',
 'rocks.prj',
 'rocks_merge.shx',
 'rocks_merge.dbf']

`rocks_merge_time_space.npy` logs the time and space usage of the instance registration process

# Evaluation

In [25]:
from rsisa.evaluation import evaluate

# note that the ground truth should be cropped shapefile instead of the original generated shapefile
evaluate('/root/rsisa/data/rocks/0_0.shp', 
         '/root/rsisa/data/rocks/rocks_merge.shp',
         IoU_threshold=0.5)

total:  11362
TP, TN, FP, FN:  3620 7576 8 158
FP:  [2579, 6095, 7016, 6270, 6489, 6712, [423, 423, 424], 6274]
FN:  [10669, 8640, 8715, 10677, 10972, 11022, 10495, 7116, 7118, 7029, 10391, 6931, 5687, 5689, 7550, 2644, 4061, 2233, 2167, 2239, 1937, 2464, 4782, 1872, 2027, 1642, 1642, 10396, 3991, 4011, 1697, 3942, 1268, 2239, 4766, 5014, 5281, 5933, 1577, 1029, 8431, 2682, 5349, 3186, 4611, 4619, 4630, 4593, 2548, 5285, 5901, 9073, 6782, 5032, 4735, 4760, 4899, 3415, 7167, 6098, 6115, 4565, 2803, 496, 3331, 3348, 9647, 547, 7829, 7439, 5493, 1043, 10930, 9276, 449, 1055, 6999, 7553, 7556, 6568, 6235, 10275, 10202, 5228, 6070, 3134, 3174, 4612, 4615, 7710, 7495, 6330, 4627, 1752, 7829, 8189, 2767, 7526, 917, 6715, 386, 8007, 7368, 850, 10306, 9080, 6667, 8566, 9287, 9233, 9431, 9135, 9138, 9139, 2060, 4697, 3464, 2927, 4079, 4080, 5714, 2013, 4254, 4266, 7827, 7821, 7321, 1844, 9240, 1082, 3531, 6793, 6868, 6869, 386, 10477, 10605, 3634, 10546, 3338, 2911, 3014, 3015, 7941, 2813, 9421,

In [28]:
from rsisa.evaluation import evaluate

# note that the ground truth should be cropped shapefile instead of the original generated shapefile
evaluate('/root/rsisa/data/filtered_0_0.shp', 
         '/root/rsisa/data/filtered_rocks_merge.shp',
         IoU_threshold=0.5)

/usr/local/lib/python3.8/dist-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)


KeyError: 11052

---

# Instance registration with instance segmentation

In [9]:
instance_dir = "/root/rsisa/rsisa/mask_rcnn/data/rock"
save_shp_file = "/root/rsisa/rsisa/mask_rcnn/data/rocks_merge.shp"
tif_map = "/root/rsisa/rsisa/mask_rcnn/data/0_0.tif"
iou_threshold = 0.5

tile_length = 10
overlap = 1
ellipse_max = 1.0
pixel_size = 0.00728542 
detection_threshold = 0.75
segmentation_threshold = 0.5

In [10]:
# instance registration
ir = Instance_Registration(instance_dir, 
                           save_shp_file,
                           tif_width_pixel=tile_length/pixel_size,
                           tif_height_pixel=tile_length/pixel_size,
                           tif_width_res=pixel_size,
                           tif_height_res=-pixel_size,
                           tif_map_file=tif_map,
                           tile_overlap_ratio=overlap/tile_length,
                           detection_threshold=detection_threshold, 
                           segmentation_threshold=segmentation_threshold, 
                           iou_threshold=iou_threshold, 
                           disable_merge=False, 
                           test=False,
                           unzip=True)

In [11]:
updated_tile_files, timestamps = ir.start_registration()
ir.combine_shapefiles()

Instance registration: 


  0%|                                                                                                | 0/360 [00:00<?, ?it/s]


IndexError: boolean index did not match indexed array along dimension 0; dimension is 36 but corresponding boolean dimension is 6